## Trabalho Final de Inteligência Artificial (INF 420)

### Tema: Pokémon Classifier

##### > Alunos: Erick Lima Figueiredo e Sávio mendes Miranda
##### > Matrículas: 98898, 98886 | Professor: Júlio Cesar Soares dos Reis

---

#### **Objetivo do Trabalho:** Produzir uma Rede Neural Convolucional para Classificação multiclasse de Pokémon.

### Bibliotecas utilizadas

In [15]:
import os
import cv2
import math
import shutil
import random
import numpy as np
import pandas as pd
import tensorflow as tf

### Configurando Comportamento da GPU

In [16]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print('Número de GPUs disponíveis: ', len(gpus))

for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

print('As GPUs foram configuradas para alocar memória à medida que for necessário.')

Número de GPUs disponíveis:  1
As GPUs foram configuradas para alocar memória à medida que for necessário.


### Obtenção e Divisão dos Dados

In [17]:
ROOT_PATH = r'D:\Documents\test'
TRAIN_PERCENTAGE = .7

In [18]:
images_train = []
images_test = []

In [19]:
folders = [os.path.join(ROOT_PATH, i) for i in os.listdir(ROOT_PATH) if os.path.isdir(os.path.join(ROOT_PATH, i))]

In [20]:
for folder in folders:
    files = os.listdir(folder)

    total_files = len(files)
    num_train = math.floor(total_files * .7)

    random.shuffle(files)

    images_train = images_train + \
        [os.path.join(folder, file) for file in files[:num_train]]
    images_test = images_test + \
        [os.path.join(folder, file) for file in files[num_train:]]


TRAIN_SIZE = len(images_train)
TEST_SIZE = len(images_test)

print(f'Existem {TRAIN_SIZE} images alocadas para treino e {TEST_SIZE} imagens alocadas para teste.')


Existem 101 images alocadas para treino e 44 imagens alocadas para teste.


#### Mover a Localização dos Dados Empregados (Dispensável se executado diretamente)

In [ ]:
TRAIN_FOLDER = os.path.join(ROOT_PATH, 'train')
TEST_FOLDER = os.path.join(ROOT_PATH, 'test')

if not os.path.isdir(TRAIN_FOLDER):
    os.mkdir(TRAIN_FOLDER)
    print(f'Diretório "train" criado em {ROOT_PATH}.')


if not os.path.isdir(TEST_FOLDER):
    os.mkdir(TEST_FOLDER)
    print(f'Diretório "test" criado em {ROOT_PATH}.')


In [ ]:
# Dispensável se está executando tudo de uma vez
for img in images_train:
    shutil.copy(img, TRAIN_FOLDER)


for img in images_test:
    shutil.copy(img, TEST_FOLDER)

print('Images copiadas!')


#### Aplicação de Hot Encoding nas Classes

In [21]:
AMOUNT_CLASSES = len(folders)

classes_reference = {}

for i, path in enumerate(folders):
    name = path.split(os.sep)[-1]
    classes_reference[name] = [0]*AMOUNT_CLASSES
    classes_reference[name][i] = 1

print(f'Hot encode completo:\n{classes_reference}')


Hot encode completo:
{'8 Bits': [1, 0, 0, 0], 'Artes minimalistas em geral': [0, 1, 0, 0], 'test': [0, 0, 1, 0], 'train': [0, 0, 0, 1]}


### Definição do Augmentador

In [39]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(.2),
    tf.keras.layers.RandomZoom(.2),
    tf.keras.layers.RandomContrast(.3)
])

### Definição dos Datasets para Execução

In [23]:
IMG_SHAPE = (64, 64, 3) # Vamos trabalhar com imagens pequenas por questões de limitação de hardware

In [24]:
BATCH_SIZE = 16

In [25]:
def generate_train_data():
    for i in range(TRAIN_SIZE):
        img = cv2.imread(images_train[i])
        img = cv2.resize(img, IMG_SHAPE) # Redimensionamento da imagem e definição de canais RGB
        img /= 255.0 # Normalização
        
        img = data_augmentation(img) # Augmentação da imagem
        
        label = images_train[i].split(os.sep)[-2]
        label = classes_reference[label] # Captura do label encodado

        yield img, label

In [ ]:
def generate_test_data():
    for i in range(TEST_SIZE):
        img = cv2.imread(images_test[i])
        img = cv2.resize(img, IMG_SHAPE) # Redimensionamento da imagem e definição de canais RGB
        img /= 255.0 # Normalização
        
        img = data_augmentation(img) # Augmentação da imagem
        
        label = images_train[i].split(os.sep)[-2]
        label = classes_reference[label] # Captura do label encodado

        yield img, label

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
    generator=generate_train_data, output_types=(tf.float64, tf.int32)).batch(BATCH_SIZE)


In [ ]:
test_dataset = tf.data.Dataset.from_generator(
    generator=generate_test_data, output_types=(tf.float64, tf.int32)).batch(BATCH_SIZE)


### Definição dos Modelos de Rede Neural para Classificação

Nosso modelo vai trabalhar com as seguintes **Funções de Ativação**:
- Rectified Linear Unit (ReLU): Retorna o valor máximo entre 0 e o tensor de entrada;
- Softmax: Mais indicada para trabalhar com modelos de classificação multiclasse (em comparação com a sigmoid), basicamente converte um vetor de valor para uma distribuição de probabilidades.

Utilizaremos um modelo sequencial composto pelos seguintes layers:
- Conv2D (Input Layer): Estabelece um núcleo de convolução alterando as dimensões do input para uma matriz 2D.
- MaxPooling2D: Reduz a entrada no quesito altura e largura, tomando o valor máximo sobre uma janela (deslocada em passos, varrendo a imagem) de entrada para cada canal da entrada.
- Flatten: Achata as dimensões do input.
- Dense (Output Layer): Dense implementa a operação: $$output = activation(dot(input, kernel) + bias)$$ onde a ativação é a função de ativação no sentido do elemento passado como argumento de ativação, kernel é uma matriz de pesos criada pela camada, e bias é um vetor de bias criado pela camada (somente aplicável se o uso_bias for Verdadeiro).

In [26]:
def create_model(using_v2=False):
    if using_v2:
        return '' # Criar um outro layout de modelo aqui
    
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (6, 6), padding='same', input_shape=IMG_SHAPE, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (4, 4), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(AMOUNT_CLASSES, activation='softmax')
    ])


In [27]:
model = create_model() # instanciamos nosso modelo


**Explicação:** Para compilar o modelo utilizaremos o otimizador Adam, que é uma abordagem mais recente e tida como mais eficiente que a abordagem de Gradiente Descendente (Gradient Descent), utilizaremos como função de avaliação de perda a `Categorical Crossentropy` que é aplicada quando existem duas ou mais classes envolvidas, é importante salientar que essa função espera labels no formato de codificação _one hot_ como já providenciamos nos códigos acima, além disso vamos usar a acurácia (proximidade entre o valor obtido experimentalmente e o valor verdadeiro) como métrica de avaliação.

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Usado ao invés do Gradiente descendente Estocástico
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])


Mais detalhes podem ser obtidos na documentação da biblioteca [Keras](https://keras.io/api/) utilizado como o backend do TensorFlow neste trabalho.

##### Visão Geral do Modelo

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 4)                

### Treinamento

In [30]:
NUM_EPOCHS = 50

In [34]:
# Aplicaremos a callback abaixo para evitar que nosso modelo entre em overfitting
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss',mode='min',verbose=1,patience=3)


In [ ]:
history = model.fit(train_dataset, epochs=NUM_EPOCHS, callbacks=[early_stop])
print('Treinamento completo =)')

In [ ]:
weights = model.get_weights()

### Avaliação e Análise de Resultados

In [ ]:
result = model.evaluate(train_dataset)

### Armazenamento do Modelo

In [ ]:
SAVE_DIR = r''

In [ ]:
model.save(SAVE_DIR+os.sep+'model.h5', save_format='h5')
print('Modelo salvo com sucesso!')
